In [25]:
%pip install nltk

NameError: ignored

In [1]:
%pip install flair

In [2]:
%pip install transformers

In [3]:
import os
import re
import numpy as np 
import pandas as pd
import random
import sklearn
from sklearn.metrics import accuracy_score

import transformers
from transformers import BertTokenizer, BertModel

import torch
from torch import cuda
from tqdm import tqdm_notebook as tqdm
device = 'cuda' if cuda.is_available() else 'cpu'

!pip3 install pickle5
import pickle5 as pickle

device

'cpu'

In [27]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from textblob import TextBlob
import torch
from flair.embeddings import BertEmbeddings
from flair.data import Sentence
import numpy as np
from collections import Counter
import seaborn as sn
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import textblob
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.models import load_model
from sklearn.metrics import precision_recall_fscore_support
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
import nltk
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

BERT = BertEmbeddings()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: Call to deprecated method __init__. (Use 'TransformerWordEmbeddings' for all transformer-based word embeddings) -- Deprecated since version 0.4.5.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a Bert

### BERT fine-tuning for document classification

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
with open('/content/drive/My Drive/SLIM/LREC/cozmo_filtered_data.pkl', "rb") as fh:
  cozmo_data = pickle.load(fh)
with open('/content/drive/My Drive/SLIM/LREC/misty_filtered_data.pkl', "rb") as fh:
  misty_data = pickle.load(fh)

In [28]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
#     text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [30]:
cozmo_data['Description'] = cozmo_data['Description'].apply(clean_text)

In [31]:
cozmo_data

,Timestamp,Description,Interst/Alarm,Confusion/Understanding,Frusteration/Relief,Sorrow/Joy,Anger/Gratitude,Fear/Hope,Boredom/Surprise,Disgust/Desire,Description_Bert
0,1.617917e+09,it went to investigate a thing it saw,1,1,3,3,3,4,4,5,"[[0.003090775, -0.33126733, -0.42522523, 0.122..."
1,1.619714e+09,drove forward for a bit and then turned to his...,2,3,3,3,3,3,3,3,"[[0.41755724, -0.010028644, 0.1145585, -0.2166..."
2,1.619714e+09,just drove forward made a little sound at the...,2,3,3,3,3,3,3,3,"[[0.38736537, -0.13056435, -0.22778009, 0.6008..."
4,1.619714e+09,looked down squinted eyes and then looked forw...,2,2,3,3,3,3,3,3,"[[0.38436273, 0.5813796, -0.046782747, -0.2924..."
5,1.619714e+09,lifted front part up moved forward and turned...,4,3,3,3,2,3,4,3,"[[0.265327, 0.18869174, 0.9075626, -0.40546942..."
...,...,...,...,...,...,...,...,...,...,...,...
648,1.637355e+09,at first the robot tilted thier head up and sa...,1,2,3,1,3,3,3,3,"[[-0.03434295, 0.5708942, -0.37369967, -0.1584..."
649,1.637355e+09,the robot first moved straight with a slight r...,1,2,1,3,1,2,3,1,"[[-0.11550679, -0.27403948, -0.63136435, 0.280..."
650,1.637355e+09,at first the robot raised their head and hand...,1,5,5,5,5,5,5,3,"[[0.20224547, 0.4459168, -0.37732568, -0.21969..."
651,1.637355e+09,the robot turned around nearly 180 degrees lo...,3,4,3,1,3,1,2,3,"[[0.34809294, 0.30013406, -0.654603, -0.123494..."


In [32]:
misty_data['Description'] = misty_data['Description'].apply(clean_text)

In [33]:
misty_data

,Timestamp,Description,Interst/Alarm,Confusion/Understanding,Frusteration/Relief,Sorrow/Joy,Anger/Gratitude,Fear/Hope,Boredom/Surprise,Disgust/Desire
1,1.617916e+09,confused and asking for clarification almost,3,2,4,3,2,4,1,1
2,1.619713e+09,his eyes half closed his head tilted up his ...,2,4,3,3,4,4,3,5
3,1.619713e+09,eyes were half open head tilted up and down s...,3,3,2,3,3,3,1,3
4,1.619713e+09,eyes were looking concerned to the left then ...,3,2,3,3,3,3,4,3
5,1.619713e+09,one arm moved about halfway up twice head tilt...,3,3,2,2,3,3,2,3
...,...,...,...,...,...,...,...,...,...,...
636,1.637353e+09,the robot moved it head up and down a few time...,2,4,5,5,5,5,4,5
638,1.637353e+09,the robot said um and raised its eyebrow and r...,4,1,1,3,1,2,5,2
639,1.637353e+09,misty raised her hed up and down a few times t...,5,1,2,3,3,1,5,3
641,1.637354e+09,at first the robot said huh and looked up and...,4,2,2,3,3,3,4,4


In [34]:
MAX_WORDS = 80
pad = lambda x: np.pad(x, [[0,MAX_WORDS - len(x[:MAX_WORDS])], [0,0]], 'constant')
mask = np.random.rand(3072) > 0

def get_embedding(sent):
    if(len(sent.strip()) == 0):
        return np.zeros((MAX_WORDS, 3072))
    sentence = Sentence(sent)
    BERT.embed(sentence)
    embed = pad(np.array([w.embedding.cpu().numpy()[mask] for w in sentence]))
    return embed

In [35]:
cozmo_data['Description_Bert'] = cozmo_data['Description'].apply(get_embedding)

In [36]:
cozmo_data

,Timestamp,Description,Interst/Alarm,Confusion/Understanding,Frusteration/Relief,Sorrow/Joy,Anger/Gratitude,Fear/Hope,Boredom/Surprise,Disgust/Desire,Description_Bert
0,1.617917e+09,it went to investigate a thing it saw,1,1,3,3,3,4,4,5,"[[0.003090775, -0.33126733, -0.42522523, 0.122..."
1,1.619714e+09,drove forward for a bit and then turned to his...,2,3,3,3,3,3,3,3,"[[0.41755724, -0.010028644, 0.1145585, -0.2166..."
2,1.619714e+09,just drove forward made a little sound at the...,2,3,3,3,3,3,3,3,"[[-0.07253877, 0.25926688, 0.045686807, 0.2180..."
4,1.619714e+09,looked down squinted eyes and then looked forw...,2,2,3,3,3,3,3,3,"[[0.27425832, 0.5068996, -0.18097569, -0.40483..."
5,1.619714e+09,lifted front part up moved forward and turned...,4,3,3,3,2,3,4,3,"[[-0.013555776, 0.18045168, 0.6174462, -0.3208..."
...,...,...,...,...,...,...,...,...,...,...,...
648,1.637355e+09,at first the robot tilted thier head up and sa...,1,2,3,1,3,3,3,3,"[[-0.029167557, 0.63686377, -0.06447758, -0.27..."
649,1.637355e+09,the robot first moved straight with a slight r...,1,2,1,3,1,2,3,1,"[[-0.0731023, -0.28304198, -0.4815937, 0.32391..."
650,1.637355e+09,at first the robot raised their head and hand...,1,5,5,5,5,5,5,3,"[[0.2112183, 0.43155318, -0.22477065, -0.46794..."
651,1.637355e+09,the robot turned around nearly 180 degrees lo...,3,4,3,1,3,1,2,3,"[[0.21229888, 0.081449635, -0.3815477, -0.2316..."


In [37]:
X = np.stack(cozmo_data.Description_Bert.values, axis=0)

# If we want to concat the word vectors:
# X = X.reshape(X.shape[:-2] + (-1,))

# If we want to sum the word vectors:
X = np.sum(X, axis=1)

Y = cozmo_data.loc[:, 'Interst/Alarm':'Disgust/Desire'].values

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
X.shape, Y.shape, X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((547, 3072), (547, 8), (492, 3072), (55, 3072), (492, 8), (55, 8))

In [55]:
model = Sequential()
model.add(Dense(1, activation='relu'))
model.add(Dense(8, activation='relu'))
model.compile(loss='kl_divergence', optimizer='adam', metrics=['accuracy'])

epochs = 800
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, verbose=1, batch_size=492)
model.summary()

Epoch 1/800
1/1 [==============================] - 1s 689ms/step - loss: 30.3301 - accuracy: 0.0589 - val_loss: 16.3506 - val_accuracy: 0.0182
Epoch 2/800
1/1 [==============================] - 0s 34ms/step - loss: 16.4658 - accuracy: 0.0305 - val_loss: 16.2136 - val_accuracy: 0.0182
Epoch 3/800
1/1 [==============================] - 0s 32ms/step - loss: 16.2365 - accuracy: 0.0305 - val_loss: 16.1743 - val_accuracy: 0.0182
Epoch 4/800
1/1 [==============================] - 0s 36ms/step - loss: 16.1882 - accuracy: 0.0305 - val_loss: 16.1549 - val_accuracy: 0.0182
Epoch 5/800
1/1 [==============================] - 0s 31ms/step - loss: 16.1685 - accuracy: 0.0305 - val_loss: 16.1438 - val_accuracy: 0.0182
Epoch 6/800
1/1 [==============================] - 0s 31ms/step - loss: 16.1573 - accuracy: 0.0305 - val_loss: 16.1350 - val_accuracy: 0.0182
Epoch 7/800
1/1 [==============================] - 0s 31ms/step - loss: 16.1502 - accuracy: 0.0305 - val_loss: 16.1303 - val_accuracy: 0.0182
Epoch

In [56]:
def predict_affect(new_desc, print_result=False, latex_format=False):
    new_embed = np.expand_dims(get_embedding(clean_text(new_desc)), axis=0)
    # IF CONCAT
    # new_embed = new_embed.reshape(new_embed.shape[:-2] + (-1,))
    # padded = np.zeros((1, 887))
    # padded[:new_embed.shape[0],:new_embed.shape[1]] = new_embed
    # new_embed = padded

    # IF SUM
    new_embed = np.sum(new_embed, axis=1)

    pred = model.predict(new_embed)
    labels =cozmo_data.loc[:, 'Interst/Alarm':'Disgust/Desire'].columns
    
    if not print_result:
        return dict(sorted(zip(labels, pred[0]), key=lambda x: x[1], reverse=True))
    
    if not latex_format:
        print(new_desc, "->", labels[np.argmax(pred)])
        for l, p in sorted(zip(labels, pred[0]), key=lambda x: x[1], reverse=True)[:3]:
            print("\t{:10}   {:0.3f}".format(l[:10], p))
        print()
    else:
        print("{} & {} \\\\".format(new_desc, labels[np.argmax(pred)]))
#         print("{} & {} ({:0.1f}) & {} ({:0.2f}) \\\\".format(new_desc, top[0][0], top[0][1],
#                                                           top[1][0], top[1][1]))

In [57]:
predict_affect("the robot squeals", print_result=True, latex_format=True)

the robot squeals & Anger/Gratitude \\
